In [20]:
import paho.mqtt.client as mqtt
import paho.mqtt.publish as publish

from pymongo import MongoClient
from bson.objectid import ObjectId

import time,datetime
import os 
import schedule

import pandas as pd
import pandas_datareader.data as web
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style

from copy import deepcopy
from IPython.display import clear_output

In [14]:
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe("IOT_NEAT_TOPIC01")

def on_message(client, userdata, msg):
    print(msg.topic+" "+str(msg.payload))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.connect("120.126.136.17", 1883, 60)

0

In [19]:

topic = "t" #TOPIC name



In [109]:
conn = MongoClient('120.126.136.17')
db = conn.Tracker 
server = "120.126.136.17"
port = 1883 #port

collection = [db.prediction]
userInfo = []
for col in collection:
    cursor = col.find({})
    dfprediction = pd.DataFrame(list(cursor))
dfprediction = dfprediction.loc[(dfprediction['year'] == datetime.datetime.now().year) & 
                                (dfprediction['month'] == datetime.datetime.now().month) &
                                (dfprediction['hour'] == datetime.datetime.now().hour) &
                                (dfprediction['day'] == datetime.datetime.now().day)]

dfrealtimetotalsteps = pd.DataFrame({})
while dfprediction.shape[0] != dfrealtimetotalsteps.shape[0]:
    collection = [db.realtimetotalsteps]# db.db2, db.dn2, db.james, db.leo
    userInfo = []
    for col in collection:
        cursor = col.find({})
        dfrealtimetotalsteps = pd.DataFrame(list(cursor))
result = (dfprediction['prediction'] * datetime.datetime.now().minute / 60).values < \
                (dfrealtimetotalsteps['hourtotalstep'].values * datetime.datetime.now().minute / 60)
diff = (dfprediction['prediction'] * datetime.datetime.now().minute / 60).values \
            - (dfrealtimetotalsteps['hourtotalstep'].values * datetime.datetime.now().minute / 60)
conn.close()
lis = list(zip(dfprediction['user'].values,result,diff))
for name,result,diff in lis:
    if result == False:
        mqttc = mqtt.Client()
        mqttc.connect(server, port)
        mqttc.publish(name, "你在偷懶歐! 還差" + str(int(diff)) +"步")

,_id,day,hour,hourtotalstep,minute,month,totalsteps,user,year
0,5cd680a55b1b0b7a70371085,11,15,0.0,58,5,2617.0,leo,2019
1,5cd680a55b1b0b7a70371086,11,15,31.0,58,5,2472.0,james,2019
2,5cd680a65b1b0b7a70371087,11,15,0.0,58,5,0.0,shuan,2019
3,5cd680a75b1b0b7a70371088,11,15,0.0,58,5,0.0,t,2019
4,5cd680a75b1b0b7a70371089,11,15,0.0,58,5,5534.0,jeng,2019


In [70]:
df

,_id,day,hour,hourtotalstep,minute,month,totalsteps,user,year
0,5cd67c6b5b1b0b7a7037102b,11,15,0.0,40,5,2617.0,leo,2019
1,5cd67c745b1b0b7a7037102c,11,15,31.0,40,5,2472.0,james,2019
2,5cd67c775b1b0b7a7037102d,11,15,0.0,40,5,0.0,shuan,2019
3,5cd67c7c5b1b0b7a7037102e,11,15,0.0,40,5,0.0,t,2019
4,5cd67c825b1b0b7a7037102f,11,15,0.0,40,5,5534.0,jeng,2019


<bound method MongoClient.close of MongoClient(host=['120.126.136.17:27017'], document_class=dict, tz_aware=False, connect=True)>

In [99]:
lis

[('leo', False, 132.6),
 ('james', False, 125.66666666666666),
 ('shuan', True, -19.066666666666666),
 ('t', False, 0.0),
 ('jeng', False, 136.93333333333334)]